In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch.nn.functional as F
from torch import nn
from torch.nn import init
from torchvision import transforms
from torchvision.datasets import MNIST

### Where does the default parameter `a=math.sqrt(5)` for `kaiming_uniform_` come from?

The default init strategy for Conv2d layers in PyTorch is Kaiming uniform.

In [3]:
nn.modules.conv._ConvNd.reset_parameters??

Signature: nn.modules.conv._ConvNd.reset_parameters(self) -> None
Docstring: <no docstring>
Source:   
    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
File:      /opt/conda/lib/python3.8/site-packages/torch/nn/modules/conv.py
Type:      function


In [4]:
nn.init.kaiming_uniform_??

Signature:
nn.init.kaiming_uniform_(
    tensor,
    a=0,
    mode='fan_in',
    nonlinearity='leaky_relu',
)
Source:   
def kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    uniform distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{U}(-\text{bound}, \text{bound})` where

    .. math::
        \text{bound} = \text{gain} \times \sqrt{\frac{3}{\text{fan\_mode}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (only
            used with ``'leaky_relu'``)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weig

Interestingly, the default value assumed for the negative slope of the rectifier unit seems arbitrary. Where does `a=math.sqrt(5)` come from? Does it work well in practice? Let's find out...

First, let's prepare a dataset we can use to test our assumptions. In this case, we are going to use the popular MNIST dataset.

In [5]:
dataset = MNIST('/workspace/data/', download=True, transform=transforms.ToTensor())
dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: /workspace/data/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
n_train = 50_000
n_valid = dataset.data.shape[0] - n_train
x_train, y_train = dataset.data[:n_train, :, :].view(n_train, -1) / 255, dataset.targets[:n_train]
x_valid, y_valid = dataset.data[n_train:, :, :].view(n_valid, -1) / 255, dataset.targets[n_train:]

In [7]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1310), tensor(0.3085))

In [8]:
def normalize(x, m, s):
    return (x - m) / s

In [9]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

Since we are going to use an architecture that use convolutional layers, we are going to format the data using a `b,c,w,h` format. We know that images in the MNIST dataset are grayscale image of size 28x28. Thus, the training set should have a shape of `(50_000, 1, 28, 28)`.

In [10]:
x_train = x_train.view(-1, 1, 28, 28)
x_valid = x_valid.view(-1, 1, 28, 28)
x_train.shape, x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [11]:
n, *_ = x_train.shape
c = y_train.max() + 1
nh = 32
n, c

(50000, tensor(10))

Let's create a `Conv2d` layer, and inspect its statistics.

In [12]:
l1 = nn.Conv2d(1, nh, 5)

In [13]:
l1.weight.shape, l1.bias.shape

(torch.Size([32, 1, 5, 5]), torch.Size([32]))

The `Conv2d` layer we just created has 32 single-channel 5x5 filters, and we have one bias unit for every filter.

In [14]:
def stats(x):
    return x.mean(), x.std()

In [15]:
stats(l1.weight), stats(l1.bias)

((tensor(-0.0001, grad_fn=<MeanBackward0>),
  tensor(0.1143, grad_fn=<StdBackward0>)),
 (tensor(0.0307, grad_fn=<MeanBackward0>),
  tensor(0.1131, grad_fn=<StdBackward0>)))

The mean and std of the randomly initialized weights  and biases are centered at zero with a std around 0.11.

What about the output activation? Let's try to pass the first 100 images of the validation set through the `Conv2d` layer...

In [16]:
x = x_valid[:100]
x.shape

torch.Size([100, 1, 28, 28])

In [17]:
t = l1(x)

In [18]:
stats(t)

(tensor(0.0300, grad_fn=<MeanBackward0>),
 tensor(0.6608, grad_fn=<StdBackward0>))

The mean is close to zero, but the std is not close to 1. This is not ideal.

### Kaiming normal

Let's check if Kaiming normal works better. If you remember, Kaiming normal is designed to be used after a ReLU or Leaky ReLU layer.

In [19]:
init.kaiming_normal_??

Signature: init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')
Source:   
def kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    normal distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{N}(0, \text{std}^2)` where

    .. math::
        \text{std} = \frac{\text{gain}}{\sqrt{\text{fan\_mode}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (only
            used with ``'leaky_relu'``)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights in the
            forward pass. Choosing ``

What is `a`? The docstring says "the negative slope of the rectifier used after this layer". In other words, it is the slope of the activation function for every value below 0. For a linear function, such slope is 1. For a ReLU, the value is 0. For a Leaky ReLU, the value is generally 0.01.

![what is a](../imgs/what_is_a.png)

In this case, we are not using a non-linearity, thus a negative slope of `a=1` is appropriate.

In [20]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(0.0164, grad_fn=<MeanBackward0>),
 tensor(1.0610, grad_fn=<StdBackward0>))

The variance of the output activation is now closer to 1. This is better than what we would have by using the default init method.

One thing to notice is that when we tried Kaiming uniform, we used a value of `a=0`. Such value assumes a `ReLU` activation. Let's try to fix this mistake and set `a=1` also for Kaiming uniform. The resulting statistics for the output activation are better, although not as good as the one generated by Kaiming normal.

In [21]:
init.kaiming_uniform_(l1.weight, a=1.)
stats(l1(x))

(tensor(0.0406, grad_fn=<MeanBackward0>),
 tensor(1.1511, grad_fn=<StdBackward0>))

In [22]:
def f1(x, a=0):
    return F.leaky_relu(l1(x), a)

In [23]:
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5332, grad_fn=<MeanBackward0>),
 tensor(0.9603, grad_fn=<StdBackward0>))

The first thing to notice is that the mean is closer to 0.5 instead of 0. This is because we are using a ReLU activation, which convert every negative value to 0. Because of that, the mean will be closer to 0.5.

In [24]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2119, grad_fn=<MeanBackward0>),
 tensor(0.4146, grad_fn=<StdBackward0>))

Remember that in the default init for every `Conv2d` layer, we are using `a=math.sqrt(5)`. As you can see, this value is highly suboptimal, as it leads to a mean far from 0.5 ― remember, we are using a ReLU so the mean is shifted towards 0.5 ― and the std is far from 1.

In [25]:
l1 = nn.Conv2d(1, nh, 5)
init.kaiming_uniform_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5584, grad_fn=<MeanBackward0>),
 tensor(1.0064, grad_fn=<StdBackward0>))

Also in this case, the Kaiming uniform init performs similarly to Kaiming normal when initialized with the correct value of `a`.